In [393]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2,RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

In [394]:
df = pd.read_csv("train.csv")
new_cols = ['C' + str(i) for i in range(1, 14 + 1)] + ['X' + str(i) for i in range(1, 19 + 2)]
new_cols[-1] = 'Y'
df.columns = new_cols

features = df.loc[:, df.columns != 'Y']
numerical_features = df.select_dtypes(include=['float64']).columns
categorical_features = df.select_dtypes(include=['object']).columns
target = df['Y']

df

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,X11,X12,X13,X14,X15,X16,X17,X18,X19,Y
0,V1,V1,V1,V1,V1,V1,V2,V1,V3,V5,...,0.352,0.073,-0.092,1.098,0.034,1.160,0.401,0.037,0.216,1
1,V1,V1,V1,V1,V1,V1,V1,V1,V2,V2,...,-1.933,-0.536,-0.092,1.098,0.034,0.716,0.401,0.724,0.216,-1
2,V1,V1,V1,V1,V1,V1,V1,V1,V2,V2,...,-0.762,-0.536,-0.092,1.098,0.034,0.716,0.401,0.712,0.216,-1
3,V1,V1,V1,V1,V1,V1,V1,V1,V1,V1,...,-0.505,-0.536,-0.092,0.182,0.034,0.716,0.401,0.393,0.216,1
4,V1,V1,V1,V1,V1,V1,V1,V1,V1,V1,...,0.409,-0.536,-0.092,0.731,0.034,0.716,0.401,0.724,0.216,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,NaN,NaN,NaN,V3,V1,NaN,NaN,NaN,V1,V4,...,NaN,NaN,NaN,-2.568,0.034,NaN,0.401,1.890,0.216,1
395,NaN,NaN,NaN,V3,V2,NaN,NaN,V1,V3,V2,...,NaN,NaN,NaN,-4.401,-2.684,NaN,0.401,1.391,0.216,1
396,NaN,NaN,NaN,V3,V2,NaN,NaN,NaN,V1,V4,...,NaN,NaN,NaN,0.548,4.926,NaN,0.401,0.992,0.216,1
397,NaN,NaN,NaN,V3,V1,NaN,NaN,NaN,V1,V8,...,NaN,NaN,NaN,-1.651,3.567,NaN,0.401,0.992,0.216,1


In [395]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3)

In [396]:
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

iterative_numerical_imputer = IterativeImputer(max_iter=10)
iterative_numerical_imputer.fit(X_train[numerical_features])
X_train.loc[:, numerical_features] = iterative_numerical_imputer.transform(X_train.loc[:, numerical_features])

C:\Users\Gábor\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\impute\_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"
C:\Users\Gábor\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [397]:
categorical_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
categorical_imputer.fit(X_train[categorical_features])
X_train.loc[:, categorical_features] = categorical_imputer.transform(X_train.loc[:, categorical_features])

X_train = pd.get_dummies(X_train)

C:\Users\Gábor\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [398]:
X_train.to_csv("export.csv", index=False)

In [399]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

classifier = RandomForestClassifier(n_estimators=1000)
#classifier = ExtraTreesClassifier(n_estimators=1000)
classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000)

In [400]:
X_test.loc[:, numerical_features] = iterative_numerical_imputer.transform(X_test.loc[:, numerical_features])
X_test.loc[:, categorical_features] = categorical_imputer.transform(X_test.loc[:, categorical_features])
X_test = pd.get_dummies(X_test)

C:\Users\Gábor\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
C:\Users\Gábor\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [401]:
y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

ValueError: X has 63 features, but DecisionTreeClassifier is expecting 67 features as input.

In [391]:
def evaluate_model(X_train, y_train, X_test, y_test, model):
	cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
	#model.fit(X_train, y_train)
	return cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

models, names = list(), list()
# SVM
models.append(SVC(gamma='auto'))
names.append('SVM')
# KNN
models.append(KNeighborsClassifier())
names.append('KNN')
# Bagging
models.append(BaggingClassifier(n_estimators=1000))
names.append('BAG')
# RF
models.append(RandomForestClassifier(n_estimators=1000))
names.append(' RF')
# ET
models.append(ExtraTreesClassifier(n_estimators=1000))
names.append(' ET')

results = list()
for i in range(len(models)):
	scores = evaluate_model(X_train, y_train, X_test, y_test, models[i])
	results.append(scores)
	print('>%s %.3f (%.3f)' % (names[i], scores.mean(), scores.std()))

>SVM 0.711 (0.024)
>KNN 0.686 (0.059)
>BAG 0.773 (0.047)
> RF 0.790 (0.033)


KeyboardInterrupt: 

In [319]:
"""
clf = RandomForestClassifier()
clf.fit(encoded_features, target)

plt.figure(figsize=(20,8))
plt.bar(encoded_features.columns, clf.feature_importances_)
plt.xticks(rotation=45)
"""

'\nclf = RandomForestClassifier()\nclf.fit(encoded_features, target)\n\nplt.figure(figsize=(20,8))\nplt.bar(encoded_features.columns, clf.feature_importances_)\nplt.xticks(rotation=45)\n'

In [320]:
#GridSearchCV?
# TODO https://datascience.stackexchange.com/questions/75345/need-help-understanding-data-leakage